# Hypothesis Testing (Core)

In [1]:
#Full Name: Hannah Ploutz

## Deliverables

- Use the same project repo as parts 1-3
- Make sure the results and visualization for all 3 hypothesis are in your notebook

## Imports

In [6]:
import pandas as pd
import numpy as np
import pymysql
import os, time, json
pymysql.install_as_MySQLdb()
from sqlalchemy import create_engine
from sqlalchemy_utils import create_database, database_exists
import tmdbsimple as tmdb
from tqdm.notebook import tqdm_notebook

## Functions

In [7]:
def get_movie_with_rating(movie_id):
    #get the movie object for the current id
    movie = tmdb.Movies(movie_id)
    
    #save the .info .releases dictionaries
    info = movie.info()
    
    releases = movie.releases()
    #loop through countries in releases
    for c in releases['countries']:
        #if the country abbrev = US
        if c['iso_3166_1'] == 'US':
            #save a certification key in info with the certification
            info['certification'] = c['certification']
    return info

In [8]:
def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

## API Call

In [9]:
with open('/Users/hgplo/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
## Display the keys of the loaded dict
login.keys()

dict_keys(['api-key'])

In [10]:
tmdb.API_KEY =  login['api-key']

## Project Start

In [11]:
#defining folder path and listing the files currently in Data folder
FOLDER = "Data/Years"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

[]

In [12]:
# Load in the dataframe from project part 1 as basics:
basics = pd.read_csv('Data/title_basics.csv.gz')
basics.head(2)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0043139,movie,Life of a Beijing Policeman,Wo zhe yi bei zi,0,2013,NaN,120,"Drama,History"


## For Loop

In [13]:
#define list of years to extract from the API
YEARS_TO_GET = [2010,2011,2012,2013,2014,2015,2016,2017,2018,2019]

In [14]:
#define an errors list
errors = [ ]

In [15]:
# Start of OUTER loop
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
     #Defining the JSON file to store results for year
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    # Check if file exists
    file_exists = os.path.isfile(JSON_FILE)

    #If it does not exist: create it
    if file_exists == False:
    # save an empty dict with just "imdb_id" to the new json file.
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)
            
    #Saving new year as the current df
    df = basics.loc[ basics['startYear']==YEAR].copy()
    
    # saving movie ids to list
    movie_ids = df['tconst'].copy()
    movie_ids

    # Load existing data from json into a dataframe called "previous_df"
    previous_df = pd.read_json(JSON_FILE)
    previous_df

    # filter out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]

    #Get index and movie id from list
   
    # INNER Loop
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
        try:
            # Retrieve then data for the movie id
            temp = get_movie_with_rating(movie_id)  
            # Append/extend results to existing file using a pre-made function
            write_json(temp,JSON_FILE)
            # Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)
            
        except Exception as e:
            errors.append([movie_id, e])

    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)

YEARS:   0%|          | 0/10 [00:00<?, ?it/s]

Movies from 2010:   0%|          | 0/3857 [00:00<?, ?it/s]

Movies from 2011:   0%|          | 0/4224 [00:00<?, ?it/s]

Movies from 2012:   0%|          | 0/4515 [00:00<?, ?it/s]

Movies from 2013:   0%|          | 0/4707 [00:00<?, ?it/s]

...

Movies from 2015:   0%|          | 0/5054 [00:00<?, ?it/s]

Movies from 2016:   0%|          | 0/5250 [00:00<?, ?it/s]

Movies from 2017:   0%|          | 0/5641 [00:00<?, ?it/s]

Movies from 2018:   0%|          | 0/5774 [00:00<?, ?it/s]

Movies from 2019:   0%|          | 0/5878 [00:00<?, ?it/s]

## Load & Inspect the Data

## Combine the Data

## Export to csv.gz

In [ ]:
## Save current unfiltered dataframe to file.
merged_df.to_csv("Data/tmdb_results_combined_final.csv.gz",compression='gzip',index=False)